In [37]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import random
import pandas as pd

In [38]:
df = pd.read_csv("data/bodyPerformance.csv")

In [39]:
cols = df.select_dtypes('object').columns.tolist()

for col in cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].values)

df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,2
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,2
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1


In [40]:
X = df.drop('class', axis=1).values
Y = df['class'].values

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [42]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

In [44]:
class Classifier(nn.Module):
    def __init__(self, _dim):
        super(Classifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(_dim, _dim*3), 
            nn.ReLU(),
            nn.Linear(_dim*3, _dim*2),
            nn.ReLU(),
            nn.Linear(_dim*2, 4)    # 은닉 8 → 출력 3 (클래스 수)
        )

    def forward(self, x):
        return self.model(x)

In [45]:
model = Classifier(X_train.shape[1])

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [47]:
epochs = 200
for epoch in range(epochs):
    # ① 순전파 (Forward)
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # ② 역전파 (Backward)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # ③ 로그 출력
    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {loss.item():.4f}")

Epoch [20/200] | Loss: 0.9191
Epoch [40/200] | Loss: 0.8091
Epoch [60/200] | Loss: 0.7439
Epoch [80/200] | Loss: 0.6685
Epoch [100/200] | Loss: 0.6295
Epoch [120/200] | Loss: 0.6070
Epoch [140/200] | Loss: 0.5921
Epoch [160/200] | Loss: 0.5812
Epoch [180/200] | Loss: 0.5735
Epoch [200/200] | Loss: 0.5667


In [48]:
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)       # 테스트 데이터 예측 (logits)
    _, predicted = torch.max(outputs, 1) # 가장 높은 확률의 클래스 선택
    acc = accuracy_score(Y_test, predicted.numpy())

print(f"\n테스트 정확도: {acc*100:.2f}%")

for i in range(5):
    print(f"실제: {Y_test[i]} | 예측: {predicted[i].item()}")


테스트 정확도: 74.99%
실제: 3 | 예측: 3
실제: 0 | 예측: 0
실제: 2 | 예측: 1
실제: 2 | 예측: 2
실제: 3 | 예측: 3
